In [223]:

def find_key_value(string_to_parse, pre_string,
                   end_string='\"',
                   index=0):
    i, istring0, istring = 0, 0, 0
    while i<=index:
        istring0 += istring
        istring = string_to_parse[istring0:].find(pre_string)
        iend = string_to_parse[istring0+istring+len(pre_string):].find(end_string)
        i+=1
    print(i)
    return string_to_parse[istring0+istring+len(pre_string):istring0+istring+len(pre_string)+iend]

<td><span class="flag es"></span> <a href="rider/ruben-plaza"><span class="uppercase"> Plaza</span> Rubén</a><span class="teammob">Caisse d'Epargne </span></td>
 es


In [447]:

from bs4 import BeautifulSoup
import requests
import numpy as np


def make_soup(url):
    r = requests.get(url)
    # results
    i0 = r.text.find('<div class=\"resultCont \"')
    results_soup = BeautifulSoup(r.text[i0:], 'html.parser')
    # stage details
    i0 = r.text.find('<div class=\"entry race\"')
    stage_soup = BeautifulSoup(r.text[i0:i0+500], 'html.parser')
    
    return results_soup, stage_soup


def find_value(string_to_parse, pre_string,
                   end_string='\"',
                   index=0):
    i, istring0, istring =0, 0, 0
    while i<=index:
        istring0 += istring
        istring = string_to_parse[istring0:].find(pre_string)
        iend = string_to_parse[istring0+istring+len(pre_string):].find(end_string)
        i+=1
    if istring>=0:
        return string_to_parse[istring0+istring+len(pre_string):istring0+istring+len(pre_string)+iend]
    else:
        return ''

def convert_string_to_seconds(time_string):
    try:
        return np.sum([(60**ii)*float(ss) for ii, ss in enumerate(time_string.split(':')[::-1]) ])
    except ValueError:
        return 0


def compute_time(time_string, prev_time):
    if time_string=='-':
        return -1
    elif prev_time==0:
        return convert_string_to_seconds(time_string)
    else:
        return convert_string_to_seconds(time_string)+prev_time

def find_periodic_lines_with_tags(soup,\
                                  tags = [{'label':'rider', 'value':'<a href="rider/'},
                                          {'label':'time', 'value':'<span class="timelag"'}], 
                                  limit=100):
    """
    look for periodic line occurence containing certain tags (e.g. 'rider', 'timelag')
    """
    ii = 0
    
    d_lines = {}
    for tag in tags:
        d_lines['start_'+tag['label']] = -1
        d_lines['period_'+tag['label']] = -1

    line_splitted = soup.find_all('td')

    for ii, l in enumerate(line_splitted[:limit]):
        for tag in tags:
            if len(str(l).split(tag['value']))>1:
                if d_lines['start_'+tag['label']]==-1:
                    d_lines['start_'+tag['label']] = ii
                elif d_lines['period_'+tag['label']]==-1:
                    d_lines['period_'+tag['label']] = ii-d_lines['start_'+tag['label']]
                else:
                    if ((ii-d_lines['start_'+tag['label']])%d_lines['period_'+tag['label']])>0:
                        print((ii-d_lines['start_'+tag['label']])%d_lines['period_'+tag['label']])
                        print('the entries do not seem periodic')
    
    output = {}
    for tag in tags:
        output['lines_'+tag['label']] = np.arange(len(line_splitted))[d_lines['start_'+tag['label']]::d_lines['period_'+tag['label']]]

    return output

def scrape_stage_details(stage_soup):
    """

    """
    line_splitted = stage_soup.find_all('td')

    stage = {}
    for ll in stage_soup.find_all('span'):
        year = find_value(str(ll), '<span class="year">', '</span>')
        if year!='':
            stage['stage_year'] = int(year)
        # edition = find_value(str(ll), '<span class="editionNr">', '</span>')
        stage_detail = find_value(str(ll), '<span class="blue">Stage ', '</span>')
        itt, ttt = stage_detail.split('(ITT)'), stage_detail.split('(TTT)')
        if len(itt)>1:
            stage['stage_number'] = int(itt[0])
            stage['stage_type'] = 'individual-time-trial'
        elif len(ttt)>1:
            stage['stage_number'] = int(ttt[0])
            stage['stage_type'] = 'team-time-trial'
        elif stage_detail!='':
            stage['stage_number'] = int(stage_detail)
            stage['stage_type'] = 'regular-stage'

        start_location = find_value(str(ll), '<span class="red">', '  › ')
        if start_location!='':
            stage['stage_start_location'] = start_location
        end_location = find_value(str(ll), ' ›  ',  '  </span>')
        if end_location!='':
            stage['stage_end_location'] = end_location
        distance = find_value(str(ll), '<span class="red distance">(',  'k)</span>')
        if distance!='':
            stage['distance_km'] = float(distance)

    return stage

def scrape_stage_results(results_soup, results_lines):
    """

    """
    line_splitted = results_soup.find_all('td')

    rank=1
    prev_time = 0
    results = {'rider_rank':[],
               'rider_id':[],
               'rider_name':[],
               'rider_nationality':[],
               'rider_stage_time_seconds':[]}

    for i1, i2 in zip(results_lines['lines_rider'], results_lines['lines_time']):
        results['rider_id'].append(find_value(str(line_splitted[i1]), '<a href=\"rider/', '\"'))
        results['rider_name'].append(find_value(str(line_splitted[i1]), '<span class="uppercase">', '</a><span').replace('</span>', ''))
        results['rider_nationality'].append(find_value(str(line_splitted[i1]), '<td><span class="flag'))
        time_string = find_value(str(line_splitted[i2]), '<span class="timeff">', '</span>')
        time = compute_time(time_string, prev_time)
        results['rider_stage_time_seconds'].append(time)
        results['rider_rank'].append(rank)
        prev_time = time
        rank+=1
    return results


results_soup, stage_soup = make_soup("https://www.procyclingstats.com/race/giro-d-italia/1999/stage-1")
results_lines = find_periodic_lines_with_tags(results_soup)
results = scrape_stage_results(results_soup, results_lines)


In [449]:

for stage in range(1,22):
    results_soup, stage_soup = make_soup("https://www.procyclingstats.com/race/giro-d-italia/1999/stage-%s"%stage)
    results_lines = find_periodic_lines_with_tags(results_soup)
    print(scrape_stage_details(stage_soup))

{'stage_year': 1999, 'stage_number': 1, 'stage_type': 'regular-stage', 'stage_start_location': 'Agrigento', 'stage_end_location': 'Modica', 'distance_km': 175.0}


{'stage_year': 1999, 'stage_number': 2, 'stage_type': 'regular-stage', 'stage_start_location': 'Noto', 'stage_end_location': 'Catania', 'distance_km': 133.0}


{'stage_year': 1999, 'stage_number': 3, 'stage_type': 'regular-stage', 'stage_start_location': 'Catania', 'stage_end_location': 'Messina', 'distance_km': 176.0}


{'stage_year': 1999, 'stage_number': 4, 'stage_type': 'regular-stage', 'stage_start_location': 'Vibo Valentia', 'stage_end_location': 'Terme Luigiane', 'distance_km': 186.0}


{'stage_year': 1999, 'stage_number': 5, 'stage_type': 'regular-stage', 'stage_start_location': 'Terme Luigiane', 'stage_end_location': 'Monte Sirino', 'distance_km': 147.0}


...

{'stage_year': 1999, 'stage_number': 17, 'stage_type': 'regular-stage', 'stage_start_location': 'Lumezzane', 'stage_end_location': 'Castelfranco Veneto', 'distance_km': 212.0}


{'stage_year': 1999, 'stage_number': 18, 'stage_type': 'individual-time-trial', 'stage_start_location': 'Treviso', 'stage_end_location': 'Treviso', 'distance_km': 45.0}


{'stage_year': 1999, 'stage_number': 19, 'stage_type': 'regular-stage', 'stage_start_location': 'Castelfranco Veneto', 'stage_end_location': 'Alpe di Pampeago', 'distance_km': 166.0}


{'stage_year': 1999, 'stage_number': 20, 'stage_type': 'regular-stage', 'stage_start_location': 'Predazzo', 'stage_end_location': 'Madonna di Campiglio', 'distance_km': 175.0}


{'stage_year': 1999, 'stage_number': 21, 'stage_type': 'regular-stage', 'stage_start_location': 'Madonna di Campiglio', 'stage_end_location': 'Aprica', 'distance_km': 190.0}


In [309]:
print(entries['rider_lines'])

[   4   13   22   31   40   49   58   67   76   85   94  103  112  121
  130  139  148  157  166  175  184  193  202  211  220  229  238  247
  256  265  274  283  292  301  310  319  328  337  346  355  364  373
  382  391  400  409  418  427  436  445  454  463  472  481  490  499
  508  517  526  535  544  553  562  571  580  589  598  607  616  625
  634  643  652  661  670  679  688  697  706  715  724  733  742  751
  760  769  778  787  796  805  814  823  832  841  850  859  868  877
  886  895  904  913  922  931  940  949  958  967  976  985  994 1003
 1012 1021 1030 1039 1048 1057 1066 1075 1084 1093 1102 1111 1120 1129
 1138 1147 1156 1165 1174 1183 1192 1201 1210 1219 1228]


In [302]:
len(soup.find_all('td'))

1233

In [294]:
for t in soup.find_all('td')[periodicity['first_time_line']::periodicity['period_time_line']][:3]:
    print(find_value(str(t), '<span class="timeff">', '</span>'))

5:32:20
0:24
0:27


In [71]:
BeautifulSoup?

In [2]:
str(soup.text.split('212927')[1][:40])

'▼2195  Gesink RobertRabobank ProTeam\xa024R'

In [26]:
x =soup.find_all(id='1')

In [48]:
i, s = 1, ['','']
while len(s)>1:
    s = soup.text.split('▼%i'%i)
    i+=1
    if len(s)>1:
        print(s[1][:20])

161  Sánchez Luis Le
91  Sastre CarlosCer
118  Rogers MichaelT
58  Vansummeren Joha
93  Gustov Volodymyr
179  Pauriol RémiCof
119  Siutsou Kanstan
83  Champion Dimitri
61 Le Mével Christop
103 de Greef Francis
61  Sánchez Luis Leó
4  Klöden AndreasRad
7  Löfkvist ThomasSk
7+1:15:52155  Gautie
4  Hesjedal RyderTea
188  Velasco IvánEus
151  Voeckler Thomas
1  Roche NicolasAG2R
178  Monier DamienCo
156  Rolland PierreB
21  Evans CadelBMC R
12  Benítez José Alb


In [40]:
i

7871